# Fake News Detection - Sequence Vectorization (Title)

## Data Preparation

In [1]:
import time

import tensorflow as tf
import tensorflow.keras.preprocessing.text as kpt
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import json
import re

import nltk
import json
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

### Import News Title as Input and News Label as Output

In [2]:
start_time = time.time()

npz_title = np.load('Title_Data.npz',allow_pickle=True)

title = npz_title['inputs']
output = npz_title['targets']

title.shape[0], output.shape[0]

(9805, 9805)

### Text Preprocessing
1. Remove Special Characters using Regular Expressions
2. Tokenize text
3. Word Normalization (Lemmatization)
4. Remove Stopwords

In [3]:
stopword = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

cleaned_title = []

for i in range(title.shape[0]):
    title_sentence = re.sub('[^A-Za-z0-9 ]+', '', title[i])
    title_tokens = nltk.word_tokenize(title_sentence)
    title_lemmatized_word = [lemmatizer.lemmatize(word) for word in title_tokens]
    title_removed_stopwords = [word for word in title_lemmatized_word if word not in stopword]
    cleaned_title.append(" ".join(word.lower() for word in title_removed_stopwords))
    
cleaned_title = np.asarray(cleaned_title,dtype=object)

len(cleaned_title), len(output)

(9805, 9805)

### Split Train and Test Data

In [4]:
X_train_title, X_test_title, y_train_title, y_test_title = train_test_split(cleaned_title, output, test_size = 0.2, random_state = 1)

### Feed News Title into Keras Tokenizer

In [5]:
max_vocab_title = 15000

tokenizer_title = Tokenizer(num_words = max_vocab_title)
tokenizer_title.fit_on_texts(X_train_title)

dictionary_title = tokenizer_title.word_index

### Convert Text to Word Sequence based on Dictionary (And Make All Texts The Same Length) 

In [6]:
def convert_text_to_index_array_title(text):
    return [dictionary_title[word] for word in kpt.text_to_word_sequence(text)]

allWordIndices_title = []

for text_title in X_train_title:
    wordIndices_title = convert_text_to_index_array_title(text_title)
    allWordIndices_title.append(wordIndices_title)

### Cast the Word Indices into Numpy Arrays (For Any Future Usage)

In [7]:
allWordIndices_title = np.asarray(allWordIndices_title)

### Create One-Hot Matrices From the Word Indices and Make the Output Categorical

In [8]:
train_title_x = tokenizer_title.sequences_to_matrix(allWordIndices_title, mode='binary')
train_title_y = tf.keras.utils.to_categorical(y_train_title, 2)

### Create and Train a Simple Neural Network Model with News Title

In [9]:
model_title = tf.keras.Sequential()

model_title.add(tf.keras.layers.Dense(64, input_shape=(max_vocab_title,), activation = 'relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(16,activation='relu'))
model_title.add(tf.keras.layers.Dropout(0.2))
model_title.add(tf.keras.layers.Dense(2,activation='relu'))
model_title.add(tf.keras.layers.Activation('sigmoid'))

model_title.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', 'Recall'])

batch_size = 500
max_epochs = 10
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model_title.fit(train_title_x, train_title_y, batch_size = batch_size, epochs = max_epochs, callbacks=[early_stopping], validation_split = 0.2 , shuffle=True, verbose=1)

Train on 6275 samples, validate on 1569 samples
Epoch 1/10
6275/6275 [==============================] - 1s 207us/sample - loss: 0.6913 - accuracy: 0.5463 - Recall: 0.7908 - val_loss: 0.6870 - val_accuracy: 0.5462 - val_Recall: 0.9669
Epoch 2/10
6275/6275 [==============================] - 1s 92us/sample - loss: 0.6759 - accuracy: 0.5941 - Recall: 0.9568 - val_loss: 0.6709 - val_accuracy: 0.5730 - val_Recall: 0.9809
Epoch 3/10
6275/6275 [==============================] - 1s 100us/sample - loss: 0.6462 - accuracy: 0.6492 - Recall: 0.9745 - val_loss: 0.6493 - val_accuracy: 0.6437 - val_Recall: 0.9720
Epoch 4/10
6275/6275 [==============================] - 1s 95us/sample - loss: 0.6050 - accuracy: 0.7131 - Recall: 0.9820 - val_loss: 0.6291 - val_accuracy: 0.6813 - val_Recall: 0.9579
Epoch 5/10
6275/6275 [==============================] - 1s 92us/sample - loss: 0.5618 - accuracy: 0.7802 - Recall: 0.9874 - val_loss: 0.6150 - val_accuracy: 0.7081 - val_Recall: 0.9382
Epoch 6/10
6275/6275 [===

### Transform Test Data into Similar Format to Feed into Neural Network Model created for News Title

In [10]:
# create another Tokenizer for model testing 
tokenizer_test_title = Tokenizer(num_words = max_vocab_title)

# this utility makes sure that all the words in your input
# are registered in the dictionary
# before trying to turn them into a matrix.
def convert_text_to_index_array_title_test(text):
    words = kpt.text_to_word_sequence(text)
    wordIndices = []
    for word in words:
        if word in dictionary_title:
            wordIndices.append(dictionary_title[word])
    return wordIndices

allWordIndices_test_title = []

for text_test in X_test_title:
    wordIndices_test = convert_text_to_index_array_title_test(text_test)
    allWordIndices_test_title.append(wordIndices_test)

# create one-hot matrices out of the indexed train_title_input
test_title_x = tokenizer_test_title.sequences_to_matrix(allWordIndices_test_title, mode='binary')

# treat the labels as categories
test_title_y = tf.keras.utils.to_categorical(y_test_title, 2)

### Test Performance on Neural Network Model (Accuracy)

In [11]:
model_title_loss, model_title_accuracy, model_title_recall = model_title.evaluate(test_title_x, test_title_y)

1961/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [12]:
model_title_loss, model_title_accuracy, model_title_recall

(0.6070780261993894, 0.7478327, 0.8980112)

In [13]:
y_pred_title = model_title.predict_classes(test_title_x)

con_mat_title = tf.math.confusion_matrix(labels=y_test_title, predictions=y_pred_title).numpy()

In [14]:
con_mat_title

array([[745, 222],
       [223, 771]])

In [15]:
end_time = time.time()

time_elapsed = end_time - start_time
time_elapsed

9.861902952194214

In [16]:
sequence_model_performance = {
                                "Title_Accuracy": str(model_title_accuracy),
                                "Title_Recall": str(model_title_recall),
                                "Time": str(time_elapsed)
                             }


with open('Model_Sequence_Title.json', 'w') as Model_Sequence_File:
    json.dump(sequence_model_performance, Model_Sequence_File)